In [1]:
#!pip install open3d

In [ ]:
#!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2

In [1]:
import open3d.ml as _ml3d
import open3d as o3d
import open3d.ml.torch as ml3d  # just switch to open3d.ml.tf for tf usage
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import torch
print(torch.cuda.is_available())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

True


In [4]:
cfg_file = "configs/randlanet_toronto3d_ADJUSTEDFORIIITB.yml"
cfg = _ml3d.utils.Config.load_from_file(cfg_file)

model = ml3d.models.RandLANet(**cfg.model)
cfg.dataset['dataset_path'] = "/mnt/d/Krishna/Research/Datasets/Toronto3D" #D:\Krishna\Research\Datasets\Toronto3D
dataset = ml3d.datasets.Toronto3D(cfg.dataset.pop('dataset_path', None), **cfg.dataset)
pipeline = ml3d.pipelines.SemanticSegmentation(model, dataset=dataset, device="gpu", **cfg.pipeline)

In [5]:
# download the weights.
ckpt_folder = "pretrained/"
os.makedirs(ckpt_folder, exist_ok=True)
ckpt_path = ckpt_folder + "randlanet_toronto3d_202201071330utc.pth"
randlanet_url = "https://storage.googleapis.com/open3d-releases/model-zoo/randlanet_toronto3d_202201071330utc.pth"
if not os.path.exists(ckpt_path):
    cmd = "wget {} -O {}".format(randlanet_url, ckpt_path)
    os.system(cmd)

# load the parameters.
pipeline.load_ckpt(ckpt_path=ckpt_path)

test_split = dataset.get_split("test")
data = test_split.get_data(0)

In [ ]:
result = pipeline.run_inference(data) # returns dict with 'predict_labels' and 'predict_scores'.
pipeline.run_test() # evaluate performance on the test set; this will write logs to './logs'.

In [6]:
import numpy as np

np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

PCfile = np.load('/mnt/d/Krishna/Research/Codes/Pretrained/logs/cache/57bc13c1925f42de3e5bac02f9406047/IIIT_Base_mSeg.ply.npy')

np.load = np_load_old

In [7]:
point = PCfile.item().get('point')
feat = PCfile.item().get('feat')
label = PCfile.item().get('label')

# Create an Open3D PointCloud object
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point)

In [8]:
# Open a text file in write mode
with open("output.txt", "w") as file:
    # Write the header
    file.write("x y z R G B label\n")

    # Iterate through each point and write the data to the file
    for i in range(point.shape[0]):
        x, y, z = point[i]
        R, G, B = feat[i]
        lbl = label[i]
        file.write(f"{x} {y} {z} {R} {G} {B} {lbl}\n")

In [9]:
!cp -r '/content/output.txt' '/content/drive/MyDrive/RandLAtest/Output'

In [2]:
!pip list

Package                   Version
------------------------- -----------
absl-py                   2.1.0
addict                    2.4.0
anyio                     4.2.0
argon2-cffi               21.3.0
argon2-cffi-bindings      21.2.0
asttokens                 2.0.5
async-lru                 2.0.4
attrs                     23.1.0
Babel                     2.11.0
beautifulsoup4            4.12.3
bleach                    4.1.0
blinker                   1.8.2
Brotli                    1.0.9
certifi                   2024.7.4
cffi                      1.16.0
charset-normalizer        3.3.2
click                     8.1.7
cmake                     3.30.1
comm                      0.2.1
ConfigArgParse            1.7
contourpy                 1.2.1
cycler                    0.12.1
dash                      2.17.1
dash-core-components      2.0.0
dash-html-components      2.0.0
dash-table                5.0.0
debugpy                   1.6.7
decorator                 5.1.1
defusedxml            